In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(
    '/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rossmann-store-sales/sample_submission.csv
/kaggle/input/rossmann-store-sales/store.csv
/kaggle/input/rossmann-store-sales/train.csv
/kaggle/input/rossmann-store-sales/test.csv


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import joblib

# loading data
train = pd.read_csv('/kaggle/input/rossmann-store-sales/train.csv', low_memory=False)
store = pd.read_csv('/kaggle/input/rossmann-store-sales/store.csv', low_memory=False)
data = pd.merge(train, store, on='Store', how='left')

# date processing
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['WeekOfYear'] = data['Date'].dt.isocalendar().week

# missing value
data.fillna(0, inplace=True)

# label encoding
le = LabelEncoder()
data['StoreType'] = le.fit_transform(data['StoreType'])
data['Assortment'] = le.fit_transform(data['Assortment'])
data['PromoInterval'] = le.fit_transform(data['PromoInterval'].astype(str))

# features and target
X = data.drop(['Sales', 'Date', 'Customers'], axis=1)
y = data['Sales']

# split training data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Onehotencode StateHoliday 
X_train = pd.get_dummies(X_train, columns=['StateHoliday'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['StateHoliday'], drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# standardlize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#turn to tensor float
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_scaled)

# data loader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# define MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 200)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# initialization model
input_size = X_train_tensor.shape[1]
model = MLP(input_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# criterion and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training 
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# predict and evaluation
model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    y_pred_tensor = model(X_test_tensor)
    y_pred = y_pred_tensor.cpu().numpy().flatten()

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MLP Evaluation -> MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}, R^2: {r2:.2f}")

Epoch [10/100], Loss: 2481272.7500
Epoch [20/100], Loss: 2822495.5000
Epoch [30/100], Loss: 4214482.5000
Epoch [40/100], Loss: 4587160.0000
Epoch [50/100], Loss: 1575234.5000
Epoch [60/100], Loss: 379559.9375
Epoch [70/100], Loss: 1985745.2500
Epoch [80/100], Loss: 1177378.6250
Epoch [90/100], Loss: 5495207.5000
Epoch [100/100], Loss: 209393.4688
MLP Evaluation -> MSE: 1682135.75, RMSE: 1296.97, MAE: 879.15, R^2: 0.89


In [3]:
feature_names = X_train.columns.tolist()
print(feature_names)

['Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Year', 'Month', 'Day', 'WeekOfYear', 'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c']


In [10]:
features = ['Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Year', 'Month', 'Day', 'WeekOfYear', 'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c']

In [12]:
# save
torch.save(model.state_dict(), 'mlp_model.pth')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(features, "features.pkl")

['features.pkl']

In [4]:
from sklearn.ensemble import RandomForestRegressor

# Train Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluation
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest - MSE: {mse_rf}, RMSE: {rmse_rf}, MAE: {mae_rf}, R^2: {r2_rf}")


Random Forest - MSE: 682784.7598545548, RMSE: 826.3079086239916, MAE: 481.7960875827017, R^2: 0.9538309439174449


In [13]:
# save
torch.save(rf_model.state_dict(), 'rf_model.pth')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(feature, "models/features.pkl")

AttributeError: 'RandomForestRegressor' object has no attribute 'state_dict'